# **IMPORT**

In [30]:
pip install scikeras

In [31]:
import keras
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import classification_report
from google.colab import drive
from keras.models import Model
import matplotlib.pyplot as plt
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Input, Dense
from keras.layers import Dense, Input
from keras.utils import to_categorical
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from scikeras.wrappers import KerasClassifier

In [32]:
# Mount Google Drive in Google Colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
# Import the Pandas library
import pandas as pd

# Specify the path to the CSV file in Google Drive and read it into a DataFrame
churning_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/CustomerChurn_dataset.csv')


# **EXPLORATORY DATA ANALYSIS**

In [34]:
churning_data

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes


In [35]:
# Display the first 5 rows of the churning_data DataFrame
churning_data.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [36]:
# Display information about the DataFrame structure, data types, and memory usage
churning_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [37]:
# Display descriptive statistics of numeric columns in the churning_data DataFrame
churning_data.describe()

,SeniorCitizen,tenure,MonthlyCharges
count,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692
std,0.368612,24.559481,30.090047
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.500000
50%,0.000000,29.000000,70.350000
75%,0.000000,55.000000,89.850000
max,1.000000,72.000000,118.750000


In [38]:
# Convert the 'TotalCharges' column in the 'churning_data' DataFrame to numeric values
churning_data['TotalCharges'] = pd.to_numeric(churning_data.TotalCharges, errors="coerce")

In [39]:
# Count the number of missing values in each column of the churning_data DataFrame
churning_data.isnull().sum()

customerID           0
gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

In [40]:
# Fill missing values in the 'TotalCharges' column of the 'churning_data' DataFrame to replace NaN values.
churning_data['TotalCharges']=churning_data['TotalCharges'].fillna(method='bfill')

In [41]:
# Display the data types of each column in the churning_data DataFrame
churning_data.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
Churn                object
dtype: object

In [42]:
# Iterate through each column in the churning_data DataFrame
for col in churning_data.columns:
    # Use pd.factorize to encode categorical values with unique integer labels, and assign to the same column
    if churning_data[col].dtype == 'object':
      churning_data[col] = pd.factorize(churning_data[col])[0]

In [43]:
# Create feature variables (X) by copying the churning_data DataFrame and dropping the "Churn" column
X = churning_data.copy().drop("Churn", axis=1)
X = X.drop("customerID", axis=1)

# Create the target variable (y) by selecting the "Churn" column
y = churning_data["Churn"]

In [44]:
scaler = MinMaxScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

scaled_churning_data = pd.concat([X_scaled, y], axis=1)


# **FEATURE SELECTION**

In [45]:
# Create a Random Forest Classifier model with 100 decision trees for feature importance
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

In [46]:
# rf_model_combined = RandomForestClassifier(n_estimators=100, random_state=42)
rfecv = RFECV(estimator=rf_model, step=1, cv=3, scoring='accuracy')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [47]:
# Fit RFECV to the training data
rfecv.fit(X_train, y_train)

RFECV(cv=3, estimator=RandomForestClassifier(random_state=42),
      scoring='accuracy')

In [48]:
# Assuming X_train is your feature matrix
selected_features = X_train.columns[rfecv.support_]

optimal_num_features = rfecv.n_features_

# Select the optimal number of features from the dataset
X = X_train[selected_features[:optimal_num_features]]


# **TRAINING**

In [49]:
X=scaled_churning_data[selected_features]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [57]:
def build_classifying_model():
    # Create a Sequential model
    model = Sequential()

    # Define the input layer with the shape of the training data
    input_layer = Input(shape=(X_train.shape[1],))

    # Add the first hidden layer with 64 units and ReLU activation
    hidden_layer_1 = Dense(64, activation='relu')(input_layer)

    # Add the second hidden layer with 32 units and ReLU activation
    hidden_layer_2 = Dense(32, activation='relu')(hidden_layer_1)

    # Add the third hidden layer with 16 units and ReLU activation
    hidden_layer_3 = Dense(16, activation='relu')(hidden_layer_2)

    # Define the output layer with 1 unit and Sigmoid activation for binary classification
    output_layer = Dense(1, activation='sigmoid')(hidden_layer_3)

    # Create a functional Model with defined input and output layers
    model = Model(inputs=input_layer, outputs=output_layer)

    # Compile the model with the specified optimizer, binary crossentropy loss, and accuracy metric
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Return the compiled model
    return model

In [59]:
# Create a KerasClassifier using the build_classifying_model function
model = KerasClassifier(build_fn=build_classifying_model)

# Define a K-Fold cross-validation strategy with 3 splits and shuffling
cv = KFold(n_splits=3, shuffle=True)

# Define a dictionary of hyperparameters for grid search
params = {
    'epochs': [10,50,100],
    'optimizer': ['adam', 'adadelta'],
}

# Create a GridSearchCV object with the KerasClassifier model and hyperparameter grid
gs = GridSearchCV(estimator=model, param_grid=params, cv=cv)

# Fit the grid search model to the training data, specifying validation data for monitoring
gs = gs.fit(X_train, y_train, validation_data=(X_test, y_test))


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


118/118 [==============================] - 1s 5ms/step - loss: 0.5011 - accuracy: 0.7463 - val_loss: 0.4202 - val_accuracy: 0.8027
Epoch 2/10
118/118 [==============================] - 0s 3ms/step - loss: 0.4396 - accuracy: 0.7902 - val_loss: 0.4077 - val_accuracy: 0.8162
Epoch 3/10
118/118 [==============================] - 0s 3ms/step - loss: 0.4352 - accuracy: 0.7886 - val_loss: 0.4064 - val_accuracy: 0.8204
Epoch 4/10
118/118 [==============================] - 0s 3ms/step - loss: 0.4306 - accuracy: 0.7937 - val_loss: 0.4078 - val_accuracy: 0.8112
Epoch 5/10
118/118 [==============================] - 0s 3ms/step - loss: 0.4288 - accuracy: 0.7913 - val_loss: 0.4069 - val_accuracy: 0.8133
Epoch 6/10
118/118 [==============================] - 0s 3ms/step - loss: 0.4288 - accuracy: 0.7894 - val_loss: 0.4060 - val_accuracy: 0.8112
Epoch 7/10
118/118 [==============================] - 0s 3ms/step - loss: 0.4250 - accuracy: 0.7942 - val_loss: 0.4050 - val_accuracy: 0.8169
Epoch 8/10
118/11

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


118/118 [==============================] - 1s 4ms/step - loss: 0.4890 - accuracy: 0.7527 - val_loss: 0.4161 - val_accuracy: 0.8048
Epoch 2/10
118/118 [==============================] - 0s 3ms/step - loss: 0.4344 - accuracy: 0.7942 - val_loss: 0.4081 - val_accuracy: 0.8119
Epoch 3/10
118/118 [==============================] - 0s 3ms/step - loss: 0.4260 - accuracy: 0.8046 - val_loss: 0.4073 - val_accuracy: 0.8062
Epoch 4/10
118/118 [==============================] - 1s 5ms/step - loss: 0.4231 - accuracy: 0.8027 - val_loss: 0.4097 - val_accuracy: 0.8169
Epoch 5/10
118/118 [==============================] - 1s 4ms/step - loss: 0.4215 - accuracy: 0.8001 - val_loss: 0.4068 - val_accuracy: 0.8062
Epoch 6/10
118/118 [==============================] - 1s 5ms/step - loss: 0.4173 - accuracy: 0.8091 - val_loss: 0.4047 - val_accuracy: 0.8176
Epoch 7/10
118/118 [==============================] - 1s 4ms/step - loss: 0.4154 - accuracy: 0.7995 - val_loss: 0.4061 - val_accuracy: 0.8219
Epoch 8/10
118/11

...

118/118 [==============================] - 2s 7ms/step - loss: 0.5243 - accuracy: 0.7159 - val_loss: 0.4143 - val_accuracy: 0.8055
Epoch 2/100
118/118 [==============================] - 1s 4ms/step - loss: 0.4365 - accuracy: 0.7883 - val_loss: 0.4112 - val_accuracy: 0.8119
Epoch 3/100
118/118 [==============================] - 1s 5ms/step - loss: 0.4311 - accuracy: 0.7942 - val_loss: 0.4124 - val_accuracy: 0.7999
Epoch 4/100
118/118 [==============================] - 1s 4ms/step - loss: 0.4256 - accuracy: 0.7934 - val_loss: 0.4057 - val_accuracy: 0.8190
Epoch 5/100
118/118 [==============================] - 1s 5ms/step - loss: 0.4200 - accuracy: 0.8032 - val_loss: 0.4070 - val_accuracy: 0.8055
Epoch 6/100
118/118 [==============================] - 1s 5ms/step - loss: 0.4244 - accuracy: 0.8030 - val_loss: 0.4146 - val_accuracy: 0.8141
Epoch 7/100
118/118 [==============================] - 1s 5ms/step - loss: 0.4177 - accuracy: 0.8051 - val_loss: 0.4045 - val_accuracy: 0.8112
Epoch 8/100

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


118/118 [==============================] - 1s 4ms/step - loss: 0.5042 - accuracy: 0.7356 - val_loss: 0.4216 - val_accuracy: 0.7984
Epoch 2/100
118/118 [==============================] - 0s 3ms/step - loss: 0.4299 - accuracy: 0.7987 - val_loss: 0.4115 - val_accuracy: 0.8112
Epoch 3/100
118/118 [==============================] - 0s 3ms/step - loss: 0.4240 - accuracy: 0.8017 - val_loss: 0.4078 - val_accuracy: 0.8183
Epoch 4/100
118/118 [==============================] - 0s 3ms/step - loss: 0.4194 - accuracy: 0.8046 - val_loss: 0.4065 - val_accuracy: 0.8197
Epoch 5/100
118/118 [==============================] - 0s 3ms/step - loss: 0.4208 - accuracy: 0.8027 - val_loss: 0.4094 - val_accuracy: 0.8091
Epoch 6/100
118/118 [==============================] - 0s 3ms/step - loss: 0.4180 - accuracy: 0.8056 - val_loss: 0.4050 - val_accuracy: 0.8126
Epoch 7/100
118/118 [==============================] - 0s 3ms/step - loss: 0.4153 - accuracy: 0.8040 - val_loss: 0.4066 - val_accuracy: 0.8062
Epoch 8/100

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


177/177 [==============================] - 2s 5ms/step - loss: 0.4638 - accuracy: 0.7650 - val_loss: 0.4198 - val_accuracy: 0.7949
Epoch 2/10
177/177 [==============================] - 1s 4ms/step - loss: 0.4304 - accuracy: 0.7982 - val_loss: 0.4098 - val_accuracy: 0.8055
Epoch 3/10
177/177 [==============================] - 1s 4ms/step - loss: 0.4245 - accuracy: 0.7987 - val_loss: 0.4063 - val_accuracy: 0.8176
Epoch 4/10
177/177 [==============================] - 1s 3ms/step - loss: 0.4210 - accuracy: 0.8012 - val_loss: 0.4042 - val_accuracy: 0.8190
Epoch 5/10
177/177 [==============================] - 0s 3ms/step - loss: 0.4181 - accuracy: 0.8042 - val_loss: 0.4061 - val_accuracy: 0.8133
Epoch 6/10
177/177 [==============================] - 0s 3ms/step - loss: 0.4159 - accuracy: 0.8023 - val_loss: 0.4085 - val_accuracy: 0.8148
Epoch 7/10
177/177 [==============================] - 1s 3ms/step - loss: 0.4157 - accuracy: 0.8058 - val_loss: 0.4067 - val_accuracy: 0.8098
Epoch 8/10
177/17

In [60]:
print("Best Parameters:", gs.best_params_)
print("Best Accuracy:", gs.best_score_)

Best Parameters: {'epochs': 10, 'optimizer': 'adam'}
Best Accuracy: 0.7990770323038694


In [62]:
# Assuming best_params contains values for 'batch_size' and 'epochs' as well
best_params = gs.best_params_

# Keras Functional API model
input_layer = Input(shape=(X_train.shape[1],))
hidden_layer_1 = Dense(64, activation='relu')(input_layer)
hidden_layer_2 = Dense(32, activation='relu')(hidden_layer_1)
hidden_layer_3 = Dense(16, activation='relu')(hidden_layer_2)
output_layer = Dense(1, activation='sigmoid')(hidden_layer_3)

# Create the best model based on the hyperparameters from grid search
best_model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model with the best optimizer (assuming 'adam' is the best optimizer)
best_model.compile(optimizer=best_params['optimizer'], loss='binary_crossentropy', metrics=['accuracy'])

# Train the model using the best hyperparameters
best_model.fit(X_train, y_train, epochs=best_params['epochs'], validation_data=(X_test, y_test))

Epoch 1/10
177/177 [==============================] - 1s 3ms/step - loss: 0.4837 - accuracy: 0.7599 - val_loss: 0.4157 - val_accuracy: 0.8133
Epoch 2/10
177/177 [==============================] - 1s 3ms/step - loss: 0.4317 - accuracy: 0.7975 - val_loss: 0.4108 - val_accuracy: 0.8077
Epoch 3/10
177/177 [==============================] - 0s 2ms/step - loss: 0.4277 - accuracy: 0.7968 - val_loss: 0.4050 - val_accuracy: 0.8226
Epoch 4/10
177/177 [==============================] - 0s 2ms/step - loss: 0.4277 - accuracy: 0.7964 - val_loss: 0.4040 - val_accuracy: 0.8190
Epoch 5/10
177/177 [==============================] - 0s 3ms/step - loss: 0.4237 - accuracy: 0.7980 - val_loss: 0.4023 - val_accuracy: 0.8183
Epoch 6/10
177/177 [==============================] - 0s 2ms/step - loss: 0.4210 - accuracy: 0.8010 - val_loss: 0.4106 - val_accuracy: 0.8133
Epoch 7/10
177/177 [==============================] - 1s 3ms/step - loss: 0.4218 - accuracy: 0.8017 - val_loss: 0.4038 - val_accuracy: 0.8133
Epoch 

In [64]:
# Get accuracy
_, accuracy = best_model.evaluate(X_train, y_train)
accuracy*100

177/177 [==============================] - 0s 2ms/step - loss: 0.4124 - accuracy: 0.8024


80.24494051933289

In [65]:
# Import the metrics module
from sklearn import metrics

# Evaluate the model and obtain predicted probabilities
y_pred = best_model.predict(X_test)
fpr_mlp, tpr_mlp, _ = metrics.roc_curve(y_test, y_pred)
auc_mlp = round(metrics.roc_auc_score(y_test, y_pred), 4)
print("AUC:", auc_mlp)

# Assuming you want to round the predicted values to integers (0 or 1)
y_pred = np.round(best_model.predict(X_test)).ravel()


45/45 [==============================] - 0s 1ms/step
AUC: 0.8577
45/45 [==============================] - 0s 2ms/step


#**OPTIMIZATION**

In [76]:
from keras.models import Model
from keras.layers import Input, Dense, Dropout
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV, KFold
from sklearn import metrics
import numpy as np

dropout_rates = [0.5, 0.7]
best_optimized_accuracy = 0.0
best_optimized_params = {}

def build_classifying_model(dropout_rate=0.3, optimizer='adam'):
  input_layer = Input(shape=(X_train.shape[1],))
  hidden_layer_1 = Dense(64, activation='relu')(input_layer)
  hidden_layer_2 = Dense(32, activation='relu')(hidden_layer_1)
  hidden_layer_3 = Dense(16, activation='relu')(hidden_layer_2)
  output_layer = Dense(1, activation='sigmoid')(hidden_layer_3)

  model = Model(inputs=input_layer, outputs=output_layer)

  # Compile the model with the given optimizer
  model.compile(optimizer=best_params['optimizer'], loss='binary_crossentropy', metrics=['accuracy'])

  # Use best_params for other parameters like batch_size and epochs
  model.fit(X_train, y_train, epochs=best_params['epochs'], validation_data=(X_test, y_test))

  return model

model = KerasClassifier(build_fn=build_classifying_model)
cv = KFold(n_splits=5, shuffle=True)

params = {
    'batch_size':[16,32,64]}

optimized_gs = GridSearchCV(estimator=model, param_grid=params, cv=cv)
optimized_gs.fit(X_train, y_train, validation_data=(X_test, y_test))

# Evaluate the model and obtain accuracy
accuracy = optimized_gs.best_score_

# Check if the current model has a higher accuracy
if accuracy > best_optimized_accuracy:
    best_optimized_accuracy = accuracy
    best_optimized_params = optimized_gs.best_params_

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


177/177 [==============================] - 2s 5ms/step - loss: 0.4854 - accuracy: 0.7542 - val_loss: 0.4171 - val_accuracy: 0.8169
Epoch 2/10
177/177 [==============================] - 1s 4ms/step - loss: 0.4303 - accuracy: 0.7994 - val_loss: 0.4045 - val_accuracy: 0.8148
Epoch 3/10
177/177 [==============================] - 1s 4ms/step - loss: 0.4249 - accuracy: 0.8000 - val_loss: 0.4054 - val_accuracy: 0.8169
Epoch 4/10
177/177 [==============================] - 1s 4ms/step - loss: 0.4232 - accuracy: 0.8016 - val_loss: 0.4031 - val_accuracy: 0.8148
Epoch 5/10
177/177 [==============================] - 1s 4ms/step - loss: 0.4224 - accuracy: 0.7968 - val_loss: 0.4054 - val_accuracy: 0.8176
Epoch 6/10
177/177 [==============================] - 1s 4ms/step - loss: 0.4191 - accuracy: 0.8000 - val_loss: 0.4041 - val_accuracy: 0.8077
Epoch 7/10
177/177 [==============================] - 1s 4ms/step - loss: 0.4179 - accuracy: 0.8028 - val_loss: 0.4086 - val_accuracy: 0.8204
Epoch 8/10
177/17

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


177/177 [==============================] - 1s 3ms/step - loss: 0.4729 - accuracy: 0.7657 - val_loss: 0.4065 - val_accuracy: 0.8190
Epoch 2/10
177/177 [==============================] - 0s 2ms/step - loss: 0.4282 - accuracy: 0.7977 - val_loss: 0.4100 - val_accuracy: 0.8169
Epoch 3/10
177/177 [==============================] - 1s 3ms/step - loss: 0.4248 - accuracy: 0.7969 - val_loss: 0.4049 - val_accuracy: 0.8162
Epoch 4/10
177/177 [==============================] - 0s 2ms/step - loss: 0.4215 - accuracy: 0.7968 - val_loss: 0.4048 - val_accuracy: 0.8098
Epoch 5/10
177/177 [==============================] - 0s 3ms/step - loss: 0.4189 - accuracy: 0.8026 - val_loss: 0.4050 - val_accuracy: 0.8141
Epoch 6/10
177/177 [==============================] - 1s 3ms/step - loss: 0.4204 - accuracy: 0.8028 - val_loss: 0.4031 - val_accuracy: 0.8155
Epoch 7/10
177/177 [==============================] - 1s 4ms/step - loss: 0.4164 - accuracy: 0.8037 - val_loss: 0.4022 - val_accuracy: 0.8148
Epoch 8/10
177/17

...

177/177 [==============================] - 2s 6ms/step - loss: 0.4854 - accuracy: 0.7561 - val_loss: 0.4123 - val_accuracy: 0.8119
Epoch 2/10
177/177 [==============================] - 1s 4ms/step - loss: 0.4302 - accuracy: 0.7918 - val_loss: 0.4072 - val_accuracy: 0.8176
Epoch 3/10
177/177 [==============================] - 1s 4ms/step - loss: 0.4258 - accuracy: 0.7948 - val_loss: 0.4064 - val_accuracy: 0.8133
Epoch 4/10
177/177 [==============================] - 1s 4ms/step - loss: 0.4247 - accuracy: 0.7975 - val_loss: 0.4042 - val_accuracy: 0.8183
Epoch 5/10
177/177 [==============================] - 1s 3ms/step - loss: 0.4230 - accuracy: 0.8017 - val_loss: 0.4032 - val_accuracy: 0.8091
Epoch 6/10
177/177 [==============================] - 0s 3ms/step - loss: 0.4201 - accuracy: 0.8010 - val_loss: 0.4054 - val_accuracy: 0.8155
Epoch 7/10
177/177 [==============================] - 0s 3ms/step - loss: 0.4197 - accuracy: 0.7998 - val_loss: 0.4024 - val_accuracy: 0.8148
Epoch 8/10
177/17

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


177/177 [==============================] - 2s 4ms/step - loss: 0.5047 - accuracy: 0.7535 - val_loss: 0.4133 - val_accuracy: 0.8041
Epoch 2/10
177/177 [==============================] - 1s 3ms/step - loss: 0.4317 - accuracy: 0.7906 - val_loss: 0.4041 - val_accuracy: 0.8183
Epoch 3/10
177/177 [==============================] - 1s 3ms/step - loss: 0.4270 - accuracy: 0.7939 - val_loss: 0.4064 - val_accuracy: 0.8119
Epoch 4/10
177/177 [==============================] - 1s 3ms/step - loss: 0.4243 - accuracy: 0.7994 - val_loss: 0.4024 - val_accuracy: 0.8197
Epoch 5/10
177/177 [==============================] - 0s 3ms/step - loss: 0.4225 - accuracy: 0.8001 - val_loss: 0.4003 - val_accuracy: 0.8141
Epoch 6/10
177/177 [==============================] - 0s 3ms/step - loss: 0.4188 - accuracy: 0.8030 - val_loss: 0.4035 - val_accuracy: 0.8112
Epoch 7/10
177/177 [==============================] - 1s 3ms/step - loss: 0.4159 - accuracy: 0.8064 - val_loss: 0.4017 - val_accuracy: 0.8204
Epoch 8/10
177/17

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


177/177 [==============================] - 2s 6ms/step - loss: 0.4690 - accuracy: 0.7634 - val_loss: 0.4116 - val_accuracy: 0.8084
Epoch 2/10
177/177 [==============================] - 1s 4ms/step - loss: 0.4313 - accuracy: 0.7898 - val_loss: 0.4051 - val_accuracy: 0.8133
Epoch 3/10
177/177 [==============================] - 1s 4ms/step - loss: 0.4247 - accuracy: 0.7987 - val_loss: 0.4091 - val_accuracy: 0.8098
Epoch 4/10
177/177 [==============================] - 1s 3ms/step - loss: 0.4241 - accuracy: 0.8032 - val_loss: 0.4037 - val_accuracy: 0.8098
Epoch 5/10
177/177 [==============================] - 0s 3ms/step - loss: 0.4186 - accuracy: 0.8039 - val_loss: 0.4009 - val_accuracy: 0.8155
Epoch 6/10
177/177 [==============================] - 1s 3ms/step - loss: 0.4181 - accuracy: 0.8016 - val_loss: 0.4036 - val_accuracy: 0.8084
Epoch 7/10
177/177 [==============================] - 1s 3ms/step - loss: 0.4183 - accuracy: 0.8033 - val_loss: 0.4021 - val_accuracy: 0.8119
Epoch 8/10
177/17

In [77]:
# Assuming best_params contains values for 'batch_size' and 'epochs' as well
best_optimized_params = optimized_gs.best_params_
best_optimized_params

{'batch_size': 64}

In [78]:

# Keras Functional API model
input_layer = Input(shape=(X_train.shape[1],))
hidden_layer_1 = Dense(64, activation='relu')(input_layer)
hidden_layer_2 = Dense(32, activation='relu')(hidden_layer_1)
hidden_layer_3 = Dense(16, activation='relu')(hidden_layer_2)
output_layer = Dense(1, activation='sigmoid')(hidden_layer_3)

best_optimized_model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model with the best optimizer (assuming 'adam' is the best optimizer)
best_optimized_model.compile(optimizer=best_params['optimizer'], loss='binary_crossentropy', metrics=['accuracy'])

# Use best_params for other parameters like batch_size and epochs
best_optimized_model.fit(X_train, y_train, batch_size=best_optimized_params['batch_size'], epochs=best_params['epochs'], validation_data=(X_test, y_test))


Epoch 1/10
89/89 [==============================] - 1s 5ms/step - loss: 0.4938 - accuracy: 0.7529 - val_loss: 0.4278 - val_accuracy: 0.7899
Epoch 2/10
89/89 [==============================] - 0s 3ms/step - loss: 0.4373 - accuracy: 0.7909 - val_loss: 0.4108 - val_accuracy: 0.8119
Epoch 3/10
89/89 [==============================] - 0s 3ms/step - loss: 0.4303 - accuracy: 0.7943 - val_loss: 0.4079 - val_accuracy: 0.8176
Epoch 4/10
89/89 [==============================] - 0s 3ms/step - loss: 0.4287 - accuracy: 0.7985 - val_loss: 0.4060 - val_accuracy: 0.8169
Epoch 5/10
89/89 [==============================] - 0s 3ms/step - loss: 0.4253 - accuracy: 0.7978 - val_loss: 0.4077 - val_accuracy: 0.8105
Epoch 6/10
89/89 [==============================] - 0s 3ms/step - loss: 0.4233 - accuracy: 0.8007 - val_loss: 0.4056 - val_accuracy: 0.8133
Epoch 7/10
89/89 [==============================] - 0s 3ms/step - loss: 0.4218 - accuracy: 0.8000 - val_loss: 0.4021 - val_accuracy: 0.8219
Epoch 8/10
89/89 [==

In [81]:
# Get accuracy
_, accuracy = best_optimized_model.evaluate(X_train, y_train)
accuracy*100

177/177 [==============================] - 0s 2ms/step - loss: 0.4130 - accuracy: 0.8046


80.45793175697327

# **TESTING**

In [82]:
loss, accuracy = best_optimized_model.evaluate(X_test, y_test)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy*100:.4f}')

45/45 [==============================] - 0s 2ms/step - loss: 0.4028 - accuracy: 0.8126
Test Loss: 0.4028
Test Accuracy: 81.2633


In [85]:
# Evaluate the model and obtain predicted probabilities
y_pred = best_optimized_model.predict(X_test)
fpr_mlp, tpr_mlp, _ = metrics.roc_curve(y_test, y_pred)
auc_mlp = round(metrics.roc_auc_score(y_test, y_pred), 4)
print("AUC:", auc_mlp)

# Assuming you want to round the predicted values to integers (0 or 1)
y_pred = np.round(best_optimized_model.predict(X_test)).ravel()

45/45 [==============================] - 0s 1ms/step
AUC: 0.8582
45/45 [==============================] - 0s 1ms/step



# **DEPLOYMENT**

In [87]:
import joblib

# With 'best_optimized_model' is your trained model
model_path = '/content/drive/My Drive/Colab Notebooks/best_opimized.model.joblib'

# Save the model to the specified file path
joblib.dump(best_optimized_model, model_path)


['/content/drive/My Drive/Colab Notebooks/best_opimized.model.joblib']